In [1]:
import pandas as pd
from deep_translator import GoogleTranslator
from spellchecker import SpellChecker
import numpy as np

# Forced-choice 

In [2]:
df_esp = pd.read_csv('../clean_data_mturk_espanol/forced_choice_emotion_mturk_espanol.csv')
df_labels_esp = pd.read_csv('../data_mturk_espanol/emotion_labels_espanol.csv')

## Manual translation

**The VADER sentiment analyzer only works in English**. I had two options:

- Using a bilingual sentiment analyzer
- Translating to English and then use VADER 

Since I couldn't find a "widely used" bilingual sentiment analyzer, and translating was straightforward, I opted fot the latter 

In [3]:
def translation_emotions(df):
    df.replace(['Enfado'], 'Anger', inplace=True)
    df.replace(['Felicidad'], 'Happiness', inplace=True)
    df.replace(['Sorpresa'], 'Surprise', inplace=True)
    df.replace(['Tristeza'], 'Sadness', inplace=True)
    df.replace(['Asco'], 'Disgust', inplace=True)
    df.replace(['Incertidumbre'], 'Uncertain', inplace=True)
    df.replace(['Miedo'], 'Fear', inplace=True)
    df.replace(['Otra'], 'Other', inplace=True)
    
    return df

In [4]:
df = translation_emotions(df_esp).copy(deep=True)
df_labels = df_labels_esp.copy(deep=True)

In [5]:
df_emo_answers_wide = df.loc[:, 'Q2.1':'Q195.1'] # subset photos

## Long format DF

In [6]:
df_emo_answers_long = df_emo_answers_wide.reset_index()
df_emo_answers_long['index'] = df_emo_answers_long.index
df_emo_answers_long = df_emo_answers_long.rename(columns={'index':'participantId'})
df_emo_answers_long.head(2)

,participantId,Q2.1,Q3.1,Q4.1,Q5.1,Q6.1,Q7.1,Q8.1,Q9.1,Q10.1,...,Q186.1,Q187.1,Q188.1,Q189.1,Q190.1,Q191.1,Q192.1,Q193.1,Q194.1,Q195.1
0,0,Other,Uncertain,Anger,Other,Anger,Anger,Other,Anger,Disgust,...,Surprise,Neutral,Other,Uncertain,Disgust,Neutral,Other,Other,Uncertain,Neutral
1,1,Neutral,Anger,Anger,Uncertain,Anger,Anger,Neutral,Other,Uncertain,...,Surprise,Neutral,Other,Happiness,Uncertain,Neutral,Uncertain,Other,Other,Neutral


In [7]:
df_labels['photoId'] =  df_labels['photoId'].str.strip()
photoIds = df_labels['photoId'].tolist()

In [8]:
df_emo_answers_long = pd.melt(df_emo_answers_long, id_vars=['participantId'], value_vars=photoIds).reset_index(drop=True).sort_values(by=['participantId'])

In [9]:
df_emo_answers_long = df_emo_answers_long.rename(columns={'variable':'photoId', 'value':'emotion'})

In [10]:
df_emo_answers_long_m = pd.merge(df_emo_answers_long, df_labels, how="left", on=["photoId"], validate="many_to_one")

In [11]:
df_emo_answers_long_m.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url
0,0,Q2.1,Other,bipoc,female,adult,enfado,https://uwmadison.co1.qualtrics.com/ControlPan...
1,0,Q99.1,Neutral,bipoc,female,adult,neutral,https://uwmadison.co1.qualtrics.com/ControlPan...


## Sentiment score

In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [13]:
sia = SentimentIntensityAnalyzer()

In [14]:
df_emo_answers_long_m['sentimentScore'] = df_emo_answers_long_m['emotion'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [15]:
df_emo_answers_long_m.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url,sentimentScore
0,0,Q2.1,Other,bipoc,female,adult,enfado,https://uwmadison.co1.qualtrics.com/ControlPan...,0.0
1,0,Q99.1,Neutral,bipoc,female,adult,neutral,https://uwmadison.co1.qualtrics.com/ControlPan...,0.0


## Center predictors

In [16]:
df_emo_answers_long_m['sexC'] = df_emo_answers_long_m['sex'].replace({'female': -0.5, 'male': 0.5})
df_emo_answers_long_m['ageC'] = df_emo_answers_long_m['age'].replace({'child': -0.5, 'adult': 0.5})
df_emo_answers_long_m['ethnicityC'] = df_emo_answers_long_m['ethnicity'].replace({'bipoc': -0.5, 'white': 0.5})

In [17]:
df_emo_answers_long_m.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url,sentimentScore,sexC,ageC,ethnicityC
0,0,Q2.1,Other,bipoc,female,adult,enfado,https://uwmadison.co1.qualtrics.com/ControlPan...,0.0,-0.5,0.5,-0.5
1,0,Q99.1,Neutral,bipoc,female,adult,neutral,https://uwmadison.co1.qualtrics.com/ControlPan...,0.0,-0.5,0.5,-0.5


In [18]:
df_emo_answers_long_m_fil = df_emo_answers_long_m[df_emo_answers_long_m['label'] != 'attention']
df_emo_answers_long_m_fil = df_emo_answers_long_m_fil.reset_index(drop=True)

In [19]:
df_emo_answers_long_m_fil.to_csv('../clean_data_mturk/forced_choice_emotion_mturk_long_format_lmer.csv', index=False)

In [20]:
df_emo_answers_long_m_fil.head()

,participantId,photoId,emotion,ethnicity,sex,age,label,url,sentimentScore,sexC,ageC,ethnicityC
0,0,Q2.1,Other,bipoc,female,adult,enfado,https://uwmadison.co1.qualtrics.com/ControlPan...,0.0000,-0.5,0.5,-0.5
1,0,Q99.1,Neutral,bipoc,female,adult,neutral,https://uwmadison.co1.qualtrics.com/ControlPan...,0.0000,-0.5,0.5,-0.5
2,0,Q98.1,Happiness,white,male,child,felicidad,https://uwmadison.co1.qualtrics.com/ControlPan...,0.5574,0.5,-0.5,0.5
3,0,Q97.1,Neutral,white,male,adult,felicidad,https://uwmadison.co1.qualtrics.com/ControlPan...,0.0000,0.5,0.5,0.5
4,0,Q96.1,Happiness,white,female,child,felicidad,https://uwmadison.co1.qualtrics.com/ControlPan...,0.5574,-0.5,-0.5,0.5


# Free-labeling

In [21]:
df_free_esp = pd.read_csv('../clean_data_mturk_espanol/free_choice_emotion_mturk_espanol.csv')
df_labels_free = pd.read_csv('../data_mturk_espanol/emotion_labels_free_choice_mturk_espanol.csv')

In [22]:
df_emo_answers_free = df_free_esp.loc[:, 'Q2.1_1':'Q195.1_4'] # get cols with words only

## Manual word clean up

In [23]:
df_emo_answers_free = df_emo_answers_free.transform(lambda x: x.str.strip(), axis = 1)
df_emo_answers_free = df_emo_answers_free.transform(lambda x: x.str.lower(), axis = 1)
df_emo_answers_free = df_emo_answers_free.transform(lambda x: x.replace({'na':'none'}) , axis = 1)
df_emo_answers_free = df_emo_answers_free.fillna('NAN')

In [24]:
df_emo_answers_free.head(2)

,Q2.1_1,Q2.1_2,Q2.1_3,Q2.1_4,Q3.1_1,Q3.1_2,Q3.1_3,Q3.1_4,Q4.1_1,Q4.1_2,...,Q193.1_3,Q193.1_4,Q194.1_1,Q194.1_2,Q194.1_3,Q194.1_4,Q195.1_1,Q195.1_2,Q195.1_3,Q195.1_4
0,molesta,NAN,NAN,NAN,molesta,NAN,NAN,NAN,molesto,NAN,...,NAN,NAN,molesto,NAN,NAN,NAN,molesto,NAN,NAN,NAN
1,enfado,NAN,NAN,NAN,enfado,NAN,NAN,NAN,enfado,NAN,...,NAN,NAN,NAN,NAN,NAN,NAN,neutral,NAN,NAN,NAN


## English translation

In [25]:
df_stack = df_emo_answers_free.stack().reset_index(drop=True) # stack as series
df_stack = df_stack.to_frame(name='emotion') # as DF
emotion_set = list(set(df_stack['emotion'])) # get distinc words

In [27]:
## dic for translation
emotion_dic = {}
for i in emotion_set:
    emotion_dic[i] = GoogleTranslator(source='spanish', target='english').translate(i)
    print(emotion_dic[i])


pride
celebration
distracted
resentment
funny
smiling
malevolent
skepticism
terrified
scared
hope
curiosity
approval
delusion
complacency
protester
bad move
grief
I long
Print
concern
intrigued
joker
satisfaction
indifference
insecurity
immaturity
expectation
afraid
suspicion
attentive
unpleasant
repentance
whim
sadness
serene
insolent
surprised
admiration
que
reflection
asco
sad
crying
complicity
protest
stubborn
seriousness
performance
repellency
notice
comfort
questioning
irreverence
nervousness
remembering
fascination
funny
tantrum
copy
depressed
captivated
intrigue
surprise
surprise
scare
astonishment
excitement
none
effusiveness
unhappy
laughing
delicious
taste
He shouted
peaceful
inconvenience
ecstasy
rude
deep
distracted
persuade
I disapprove
disgusted
I didn't expect it from you
sarcasm
reflective
missed
pensive
clarity
naughty
aggressiveness
attention
notch
disgust
happiness
cooperation
containment
excited
intuition
annoying
melancholia
smug
denial
horrified
provocation
neutr

In [28]:
df_emo_answers_free_en = df_emo_answers_free.replace(emotion_dic)

In [29]:
df_labels_free['photoId'] = df_emo_answers_free.columns.tolist()
df_labels_free.tail(2)

,ethnicity,sex,age,label,url,photoId
774,white,male,child,incertidumbre,https://uwmadison.co1.qualtrics.com/ControlPan...,Q195.1_3
775,white,male,child,incertidumbre,https://uwmadison.co1.qualtrics.com/ControlPan...,Q195.1_4


## Long-form DF

In [30]:
df_emo_answers_free_long = df_emo_answers_free_en.reset_index()
df_emo_answers_free_long['index'] = df_emo_answers_free_long.index
df_emo_answers_free_long = df_emo_answers_free_long.rename(columns={'index':'participantId'})
df_emo_answers_free_long.head(2)

,participantId,Q2.1_1,Q2.1_2,Q2.1_3,Q2.1_4,Q3.1_1,Q3.1_2,Q3.1_3,Q3.1_4,Q4.1_1,...,Q193.1_3,Q193.1_4,Q194.1_1,Q194.1_2,Q194.1_3,Q194.1_4,Q195.1_1,Q195.1_2,Q195.1_3,Q195.1_4
0,0,annoying,nan,nan,nan,annoying,nan,nan,nan,upset,...,nan,nan,upset,nan,nan,nan,upset,nan,nan,nan
1,1,anger,nan,nan,nan,anger,nan,nan,nan,anger,...,nan,nan,nan,nan,nan,nan,neutral,nan,nan,nan


In [31]:
# df_labels['photoId'] =  df_labels['photoId'].str.strip()
photoIds = df_labels_free['photoId'].tolist()

In [32]:
df_emo_answers_free_long = pd.melt(df_emo_answers_free_long, id_vars=['participantId'], value_vars=photoIds).reset_index(drop=True).sort_values(by=['participantId'])

In [33]:
df_emo_answers_free_long = df_emo_answers_free_long.rename(columns={'variable':'photoId', 'value':'emotion'})

In [34]:
## replace with np.nan to drop it 
dic_nan = {'nan': np.nan, 'none': np.nan}

df_emo_answers_free_long['emotion'] = df_emo_answers_free_long['emotion'].replace(dic_nan)

In [35]:
df_emo_answers_free_long.head()

,participantId,photoId,emotion
0,0,Q2.1_1,annoying
14812,0,Q163.1_1,happy
4255,0,Q48.1_2,NaN
4232,0,Q48.1_1,upset
14835,0,Q163.1_2,happy


## Check and drop NA

In [36]:
df_emo_answers_free_long.isna().sum(), df_emo_answers_free_long.shape

(participantId        0
 photoId              0
 emotion          11883
 dtype: int64,
 (17848, 3))

In [37]:
## drop np.nan
df_emo_answers_free_long_drop = df_emo_answers_free_long.dropna()

df_emo_answers_free_long_drop.isna().sum(), df_emo_answers_free_long_drop.shape

(participantId    0
 photoId          0
 emotion          0
 dtype: int64,
 (5965, 3))

In [38]:
df_emo_answers_free_long_m = pd.merge(df_emo_answers_free_long_drop, df_labels_free, how="left", on=["photoId"], validate="many_to_one")

In [39]:
df_emo_answers_free_long_m.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url
0,0,Q2.1_1,annoying,bipoc,female,adult,enfado,https://uwmadison.co1.qualtrics.com/ControlPan...
1,0,Q163.1_1,happy,white,male,child,felicidad,https://uwmadison.co1.qualtrics.com/ControlPan...


In [40]:
df_emo_answers_free_long_m.isna().sum(), df_emo_answers_free_long_m.shape

(participantId     0
 photoId           0
 emotion           0
 ethnicity        72
 sex              72
 age              72
 label             0
 url               0
 dtype: int64,
 (5965, 8))

## Manual english correction and removing sentences

In [41]:
## we want to get rid of sentences and leave single words

## count length sentences 
df_emo_answers_free_long_m['len_words'] = df_emo_answers_free_long_m['emotion'].str.split().apply(len)

## get list of distinct sentences (>= 2 words)
list_sentences = df_emo_answers_free_long_m[df_emo_answers_free_long_m['len_words'] >= 2].value_counts('emotion').index.tolist()

list_sentences

['go to',
 'Calm down',
 'I laughed',
 'I long',
 'I disapprove',
 'would be',
 'Clark Kent',
 'He shouted',
 "I didn't expect it from you",
 'I knew',
 'bad attitude',
 'bad move',
 'in love',
 'man of steel in disgrace',
 'poor appetite']

In [42]:
## dic for word replacement

dic_sentences = {}

for i in list_sentences:
    dic_sentences[i] = None
    
dic_sentences

{'go to': None,
 'Calm down': None,
 'I laughed': None,
 'I long': None,
 'I disapprove': None,
 'would be': None,
 'Clark Kent': None,
 'He shouted': None,
 "I didn't expect it from you": None,
 'I knew': None,
 'bad attitude': None,
 'bad move': None,
 'in love': None,
 'man of steel in disgrace': None,
 'poor appetite': None}

In [43]:
## there is no choice but manual translation/fixing for these ones

dic_sentences['go to'] = 'wrath' ## Google Translate is spliting 'ira' (wrath) into 'ir a'
dic_sentences['Calm down'] =  'calm'
dic_sentences['I laughed'] =  'laughing'
dic_sentences['I long'] = 'longing'
dic_sentences['I disapprove'] = 'disapprove'
dic_sentences['would be'] = np.nan 
dic_sentences['Clark Kent'] = np.nan
dic_sentences['He shouted'] = 'shouting'
dic_sentences['I knew'] = 'know'
dic_sentences['bad attitude'] = 'gloomy'
dic_sentences['bad move'] = np.nan
dic_sentences['in love'] = 'love'
dic_sentences['man of steel in disgrace'] = 'disgraced'
dic_sentences['poor appetite'] = 'satiated'
dic_sentences["I didn't expect it from you"] = np.nan

In [44]:
## to search for 'keys' based on 'value'
print(list(emotion_dic.keys())[list(emotion_dic.values()).index('bad move')]) 

mala jugada


In [45]:
dic_sentences

{'go to': 'wrath',
 'Calm down': 'calm',
 'I laughed': 'laughing',
 'I long': 'longing',
 'I disapprove': 'disapprove',
 'would be': nan,
 'Clark Kent': nan,
 'He shouted': 'shouting',
 "I didn't expect it from you": nan,
 'I knew': 'know',
 'bad attitude': 'gloomy',
 'bad move': nan,
 'in love': 'love',
 'man of steel in disgrace': 'disgraced',
 'poor appetite': 'satiated'}

In [46]:
## replace with new translation

df_emo_answers_free_long_m = df_emo_answers_free_long_m.replace(dic_sentences)

In [47]:
## recheck NAN values
df_emo_answers_free_long_m.isna().sum(), df_emo_answers_free_long_m.shape

(participantId     0
 photoId           0
 emotion           5
 ethnicity        72
 sex              72
 age              72
 label             0
 url               0
 len_words         0
 dtype: int64,
 (5965, 9))

In [48]:
## drop np.nan
df_emo_answers_free_long_m = df_emo_answers_free_long_m.dropna()

df_emo_answers_free_long_m.isna().sum(), df_emo_answers_free_long_m.shape

(participantId    0
 photoId          0
 emotion          0
 ethnicity        0
 sex              0
 age              0
 label            0
 url              0
 len_words        0
 dtype: int64,
 (5888, 9))

In [49]:
#### recheck sentences

## count length sentences 
df_emo_answers_free_long_m['len_words'] = df_emo_answers_free_long_m['emotion'].str.split().apply(len)

## get list of distinct sentences (>= 2 words)
list_sentences_ = df_emo_answers_free_long_m[df_emo_answers_free_long_m['len_words'] >= 2].value_counts('emotion').index.tolist()

list_sentences_

[]

In [50]:
## remove len_words column
df_emo_answers_free_long_m = df_emo_answers_free_long_m.drop(['len_words'], axis=1)
df_emo_answers_free_long_m.head()

,participantId,photoId,emotion,ethnicity,sex,age,label,url
0,0,Q2.1_1,annoying,bipoc,female,adult,enfado,https://uwmadison.co1.qualtrics.com/ControlPan...
1,0,Q163.1_1,happy,white,male,child,felicidad,https://uwmadison.co1.qualtrics.com/ControlPan...
2,0,Q48.1_1,upset,white,male,adult,tristeza,https://uwmadison.co1.qualtrics.com/ControlPan...
3,0,Q163.1_2,happy,white,male,child,felicidad,https://uwmadison.co1.qualtrics.com/ControlPan...
4,0,Q194.1_1,upset,white,male,adult,incertidumbre,https://uwmadison.co1.qualtrics.com/ControlPan...


## Sentiment score

In [51]:
## compute sentiment score for each word
df_emo_answers_free_long_m['sentimentScore'] = df_emo_answers_free_long_m['emotion'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [52]:
df_emo_answers_free_long_m.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url,sentimentScore
0,0,Q2.1_1,annoying,bipoc,female,adult,enfado,https://uwmadison.co1.qualtrics.com/ControlPan...,-0.4019
1,0,Q163.1_1,happy,white,male,child,felicidad,https://uwmadison.co1.qualtrics.com/ControlPan...,0.5719


## Center predictors

In [53]:
df_emo_answers_free_long_m['sexC'] = df_emo_answers_free_long_m['sex'].replace({'female': -0.5, 'male': 0.5})
df_emo_answers_free_long_m['ageC'] = df_emo_answers_free_long_m['age'].replace({'child': -0.5, 'adult': 0.5})
df_emo_answers_free_long_m['ethnicityC'] = df_emo_answers_free_long_m['ethnicity'].replace({'bipoc': -0.5, 'white': 0.5})

In [54]:
df_emo_answers_free_long_m.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url,sentimentScore,sexC,ageC,ethnicityC
0,0,Q2.1_1,annoying,bipoc,female,adult,enfado,https://uwmadison.co1.qualtrics.com/ControlPan...,-0.4019,-0.5,0.5,-0.5
1,0,Q163.1_1,happy,white,male,child,felicidad,https://uwmadison.co1.qualtrics.com/ControlPan...,0.5719,0.5,-0.5,0.5


In [55]:
df_emo_answers_free_long_m_fil = df_emo_answers_free_long_m[df_emo_answers_free_long_m['label'] != 'attention']
df_emo_answers_free_long_m_fil = df_emo_answers_free_long_m_fil.reset_index(drop=True)

In [56]:
df_emo_answers_free_long_m_fil.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url,sentimentScore,sexC,ageC,ethnicityC
0,0,Q2.1_1,annoying,bipoc,female,adult,enfado,https://uwmadison.co1.qualtrics.com/ControlPan...,-0.4019,-0.5,0.5,-0.5
1,0,Q163.1_1,happy,white,male,child,felicidad,https://uwmadison.co1.qualtrics.com/ControlPan...,0.5719,0.5,-0.5,0.5


In [57]:
df_emo_answers_free_long_m_fil.to_csv('../clean_data_mturk_espanol/free_labeling_emotion_mturk_long_format_lmer_espanol.csv', index=False)